In [260]:
!pip install -q tensorflow-recommenders
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow_recommenders as tfrs
from typing import Dict, Text
import pprint

In [261]:
product_data = pd.read_csv('2021June-July_product_data.csv', on_bad_lines='warn')
product_data['product_itemid'] = product_data['product_itemid'].astype('string')
print(product_data.shape)
product_data.head(5)

(1583, 33)


,pk_product,date_collected,product_itemid,product_shopid,product_category,product_name,product_price,product_price_min,product_price_max,product_discount,...,status,shop_location,shop_is_on_flash_sale,shop_is_preferred_plus_seller,feature_lowest_price_guarantee,feature_can_use_bundle_deal,feature_can_use_cod,feature_can_use_wholesale,feature_show_free_shipping,product_variation_count
0,202106137919342724,2021-06-13,7919342724,1083822,Short Sleeves,NKD Korean Fashion Erich Basic Daily Polo Tie ...,300.0,135,138,55,...,1,"Taytay, Rizal",0,0,0,0,0,0,0,20
1,202106137140015494,2021-06-13,7140015494,130651949,Short Sleeves,OnlyYouth Classic knitted blouse korean top sh...,NaN,149,149,0,...,1,"San Nicolas, Metro Manila",0,0,0,0,0,0,0,9
2,202106134843903203,2021-06-13,4843903203,55748694,Short Sleeves,SS Oversized Ringer Top Plus Size Korean Fashi...,200.0,75,75,63,...,1,"Taytay, Rizal",0,0,0,1,0,0,0,14
3,2021061311405348262,2021-06-13,11405348262,396605392,Short Sleeves,Women's T-shirt Short Sleeve Collar with small...,391.0,294,294,25,...,1,Mainland China,0,0,0,1,0,0,0,3
4,202106132553628745,2021-06-13,2553628745,55748694,Short Sleeves,SS Korean Tops Square Neck Fashion Blouse Semi...,300.0,55,55,82,...,1,"Taytay, Rizal",0,0,1,1,0,0,1,10


In [262]:
review_data = pd.read_csv('2021June-July_review_data.csv', on_bad_lines='warn')
review_data['user_id'] = review_data.groupby(['author_username']).ngroup()
review_data['user_id'] = review_data['user_id'].astype('string')
review_data['product_itemid'] = review_data.apply(lambda x: str(x['itemid']), axis=1)
review_data = review_data.drop(['author_username', 'comment', 'itemid'], axis=1)
print(review_data.shape)
review_data.head(5)

(51279, 21)


,pk_review,date_collected,cmtid,shopid,rating_star,no_tag,pos_good_quality,pos_excellent_quality,pos_very_accomodating,pos_well_packaged,...,pos_will_order_again,neg_defective,neg_did_not_receive_item,neg_damaged_packaging,neg_will_not_order_again,neg_rude_seller,neg_item_shipped_late,neg_item_different_from_picture,user_id,product_itemid
0,202106134996449880,2021-06-13,4996449880,130651949,5,0,0,1,0,0,...,0,0,0,0,0,0,0,0,11077,3550379942
1,202106135025745783,2021-06-13,5025745783,130651949,5,0,0,1,0,0,...,1,0,0,0,0,0,0,0,599,3550379942
2,202106134994515949,2021-06-13,4994515949,130651949,5,0,0,1,0,0,...,0,0,0,0,0,0,0,0,18042,3550379942
3,202106135006142740,2021-06-13,5006142740,130651949,5,0,0,1,0,0,...,0,0,0,0,0,0,0,0,18255,3550379942
4,202106134927077288,2021-06-13,4927077288,130651949,5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,19699,3550379942


In [263]:
product_data_ds = tf.data.Dataset.from_tensor_slices(dict(product_data))

for x in product_data_ds.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'date_collected': b'2021-06-13',
 'feature_can_use_bundle_deal': 0,
 'feature_can_use_cod': 0,
 'feature_can_use_wholesale': 0,
 'feature_lowest_price_guarantee': 0,
 'feature_show_free_shipping': 0,
 'pk_product': 202106137919342724,
 'prod_rate_star_0': 3447,
 'prod_rate_star_1': 146,
 'prod_rate_star_2': 70,
 'prod_rate_star_3': 9,
 'prod_rate_star_4': 24,
 'prod_rate_star_5': 3696,
 'product_brand': b'No Brand',
 'product_category': b'Short Sleeves',
 'product_comment_count': 3696,
 'product_discount': 55,
 'product_itemid': b'7919342724',
 'product_like_count': 4504,
 'product_name': b'NKD Korean Fashion Erich Basic Daily Polo Tie Top 1104',
 'product_price': 300.0,
 'product_price_max': 138,
 'product_price_min': 135,
 'product_shopid': 1083822,
 'product_total_rating': 4.89,
 'product_variation_count': 20,
 'product_views': 12815,
 'shop_is_on_flash_sale': 0,
 'shop_is_preferred_plus_seller': 0,
 'shop_location': b'Taytay, Rizal',
 'status': 1,
 'stock': 134411,
 'units_sold': 

In [264]:
review_data_ds = tf.data.Dataset.from_tensor_slices(dict(review_data))

for x in review_data_ds.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'cmtid': 4996449880,
 'date_collected': b'2021-06-13',
 'neg_damaged_packaging': 0,
 'neg_defective': 0,
 'neg_did_not_receive_item': 0,
 'neg_item_different_from_picture': 0,
 'neg_item_shipped_late': 0,
 'neg_rude_seller': 0,
 'neg_will_not_order_again': 0,
 'no_tag': 0,
 'pk_review': 202106134996449880,
 'pos_excellent_quality': 1,
 'pos_good_quality': 0,
 'pos_item_shipped_immediately': 0,
 'pos_very_accomodating': 0,
 'pos_well_packaged': 0,
 'pos_will_order_again': 0,
 'product_itemid': b'3550379942',
 'rating_star': 5,
 'shopid': 130651949,
 'user_id': b'11077'}


In [265]:
review_data_ds = review_data_ds.map(lambda x: {
    "product_itemid": x["product_itemid"],
    "user_id": x["user_id"],
})
product_data_ds = product_data_ds.map(lambda x: x["product_itemid"])

In [266]:
tf.random.set_seed(42)
shuffled = review_data_ds.shuffle(len(train), seed=42, reshuffle_each_iteration=False)

train = shuffled.take(int(len(train)*0.8))
test = shuffled.skip(int(len(train)*0.8)).take(int(len(train)*0.2))

print(len(train))
print(len(test))

32818
6563


In [267]:
product_itemid = product_data_ds.batch(1_000)
user_id = review_data_ds.batch(10_000).map(lambda x: x["user_id"])

unique_product = np.unique(np.concatenate(list(product_itemid)))
unique_user = np.unique(np.concatenate(list(user_id)))

#unique_user = np.asarray(unique_user).astype(np.int)
print(len(product_data_ds))
print(unique_product.shape)
print(len(review_data_ds))
print(unique_user.shape)
#unique_user[:10]


1583
(1038,)
51279
(21027,)


In [268]:
embedding_dimension = 32

In [269]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(vocabulary=unique_user, mask_token=None),
  tf.keras.layers.Embedding(len(unique_user) + 1, embedding_dimension)
])

In [270]:
product_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(vocabulary=unique_product, mask_token=None),
  tf.keras.layers.Embedding(len(unique_product) + 1, embedding_dimension)
])

In [271]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=product_data_ds.batch(128).map(product_model)
)

In [272]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [273]:
class ProductModel(tfrs.Model):

  def __init__(self, user_model, product_model):
    super().__init__()
    self.product_model: tf.keras.Model = product_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    positive_product_embeddings = self.product_model(features["product_itemid"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_product_embeddings)

In [274]:
class NoBaseClassProductModel(tf.keras.Model):

  def __init__(self, user_model, product_model):
    super().__init__()
    self.product_model: tf.keras.Model = product_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Set up a gradient tape to record gradients.
    with tf.GradientTape() as tape:

      # Loss computation.
      user_embeddings = self.user_model(features["user_id"])
      positive_product_embeddings = self.product_model(features["product_itemid"])
      loss = self.task(user_embeddings, positive_product_embeddings)

      # Handle regularization losses as well.
      regularization_loss = sum(self.losses)

      total_loss = loss + regularization_loss

    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

  def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Loss computation.
    user_embeddings = self.user_model(features["user_id"])
    positive_product_embeddings = self.product_model(features["product_itemid"])
    loss = self.task(user_embeddings, positive_product_embeddings)

    # Handle regularization losses as well.
    regularization_loss = sum(self.losses)

    total_loss = loss + regularization_loss

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

In [275]:
model = ProductModel(user_model, product_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [292]:
#Probably change things in train/test split idk

cached_train = train.shuffle(len(train)).batch(int(len(train)*0.1)).cache()
cached_test = test.batch(int(len(train)*0.02)).cache()
print(cached_train)
print(cached_test)

<CacheDataset shapes: {product_itemid: (None,), user_id: (None,)}, types: {product_itemid: tf.string, user_id: tf.string}>
<CacheDataset shapes: {product_itemid: (None,), user_id: (None,)}, types: {product_itemid: tf.string, user_id: tf.string}>


In [293]:
model.fit(cached_train, epochs=3)

Epoch 1/3
11/11 [==============================] - 5s 425ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0903 - factorized_top_k/top_5_categorical_accuracy: 0.6666 - factorized_top_k/top_10_categorical_accuracy: 0.7322 - factorized_top_k/top_50_categorical_accuracy: 0.8485 - factorized_top_k/top_100_categorical_accuracy: 0.8991 - loss: 11140.8742 - regularization_loss: 0.0000e+00 - total_loss: 11140.8742
Epoch 2/3
11/11 [==============================] - 5s 418ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1099 - factorized_top_k/top_5_categorical_accuracy: 0.6736 - factorized_top_k/top_10_categorical_accuracy: 0.7388 - factorized_top_k/top_50_categorical_accuracy: 0.8594 - factorized_top_k/top_100_categorical_accuracy: 0.9083 - loss: 10231.9308 - regularization_loss: 0.0000e+00 - total_loss: 10231.9308
Epoch 3/3
11/11 [==============================] - 5s 418ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1373 - factorized_top_k/top_5_categorical_accuracy: 

In [294]:
model.evaluate(cached_test, return_dict=True)

11/11 [==============================] - 2s 141ms/step - factorized_top_k/top_1_categorical_accuracy: 0.4519 - factorized_top_k/top_5_categorical_accuracy: 0.7213 - factorized_top_k/top_10_categorical_accuracy: 0.7774 - factorized_top_k/top_50_categorical_accuracy: 0.8822 - factorized_top_k/top_100_categorical_accuracy: 0.9221 - loss: 1125.7248 - regularization_loss: 0.0000e+00 - total_loss: 1125.7248


{'factorized_top_k/top_1_categorical_accuracy': 0.4519274830818176,
 'factorized_top_k/top_5_categorical_accuracy': 0.7213164567947388,
 'factorized_top_k/top_10_categorical_accuracy': 0.7773883938789368,
 'factorized_top_k/top_50_categorical_accuracy': 0.8822184801101685,
 'factorized_top_k/top_100_categorical_accuracy': 0.9221392869949341,
 'loss': 0.22221320867538452,
 'regularization_loss': 0,
 'total_loss': 0.22221320867538452}